# Asynchronous programming with Python
## Module 5 - Compare asynchronous code approaches - multiprocessing

### Agenda:

* Multiprocessing - calculate π value with Monte-Carlo method.
    * Using ProcessPoolExecutor.
    * Trio.
    * AsyncIO.

### Calculating the π value with Monte-Carlo method

Monte Carlo methods, or Monte Carlo experiments, are a broad class of
computational algorithms that rely on repeated random sampling
to obtain numerical results.

<div align="right">
    – <a href="https://en.wikipedia.org/wiki/Monte_Carlo_method">Wikipedia / Monte Carlo method </a>
</div>

#### The method definition
<div align="center"><img src="../images/Pi_30K.gif" alt="three seagulls" width="200"/></div>

We have a square with a side of length 1, and a quadrant (circular sector) inscribed in it.

The area of the square is 1.

A circul area is calculated as $πr^{2}$, with $r=1$ it gives just $π$. So, the area of the quadrant is $\frac{π}{4}$.

If we drop random points on a square, some of them will land in the quadrant, and some of them will be outside of it.

The ratio of the points inside the quadrant and the total number of points is an estimate of the ratio of the two areas.  That is:

$$\frac{N_{quadrant}}{N_{total}} = \frac{π}{4}$$

$$π = 4 \frac{N_{quadrant}}{N_{total}}$$

All points inside the quadrant meet following condition:

$$x^{2} + y^{2} ≤ 1$$

#### Implementation

Here is a simple (and inefficient) implementation:

In [2]:
import math
import random

POINTS = 100_000_000


def get_points(n):
    """Get points inside the quadrant."""
    points = 0
    random_ = random.random

    for _ in range(int(n)):
        x = random_()
        y = random_()
        points += (x*x + y*y) <= 1

    return points


def print_result(π):
    """Print out the result of the calculated π value."""
    print(π)
    print("The difference is:", π - math.pi)

In [7]:
%%time
# This takes half a minute on my machine

print("The π is roughly equal to...")
π = 4 * get_points(POINTS) / POINTS
print_result(π)

The π is roughly equal to...
3.14148084
The difference is: -0.00011181358979328593
CPU times: user 26.3 s, sys: 221 ms, total: 26.5 s
Wall time: 27 s


👉 *What is the maximum number of random points we can use for such kind of tasks?  Python uses [Mersenne Twister](https://en.wikipedia.org/wiki/Mersenne_Twister), one of the most efficient pseudo-random generators, that has a period of $4.3✕10^{106001}$.  That is, we are good to use Python's `random` module without a risk to run out of random points.* 👈


#### Parallelization
We can have distribute the points calculation over subprocesses,
then aggregate the numbers.

We will distribute the task across 10 workers, each will provide 1/10 of points.

In [6]:
SUBTASKS_NUM = 10
SUBTASK_POINTS = POINTS / SUBTASKS_NUM

For the simplest cases you can just go with
[`ProcessPoolExecutor`](https://docs.python.org/3/library/concurrent.futures.html#processpoolexecutor),
which handles most of the subprocess-related logic for us.

It has the same interface as `ThreadPoolExecutor` discussed on a
previous module.

In [9]:
%%time
# On my 4-core machine it takes 10 seconds.

from concurrent.futures import ProcessPoolExecutor

print("The π is roughly equal to...")

with ProcessPoolExecutor() as executor:
    points = sum(
        executor.map(get_points, [SUBTASK_POINTS for _ in range(SUBTASKS_NUM)])
    )


π = 4 * points / POINTS
print_result(π)

The π is roughly equal to...
3.14169296
The difference is: 0.0001003064102067519
CPU times: user 24 ms, sys: 37.8 ms, total: 61.7 ms
Wall time: 9.07 s


### Subprocesses with `async/await` code
The example above showed how code can be executed *asynchronously*
without any async/await syntax.  Though, it's often desired for your
application to keep working with the main thread while waiting for
a response from a subprocess.

`async/await` code can help with this, also for current task is not
necessary.

Both Trio and AsyncIO use interface similar to
[subprocess.Popen](https://docs.python.org/3/library/subprocess.html#popen-constructor).

In order to pass `get_points` function to a subprocess we'll need
some common arguments.

In [16]:
import inspect
import sys


SUBTASK_SCRIPT_TEMPLATE = """
import random

{func_src}

print(get_points({points_num}))
"""


def get_subprocess_arguments():
    """Provide arguments to run `get_points` from a subprocess.

    The arguments are equivalent to calling a subprocess with
    a shell command

        python -c 'import random
        def get_points(n):
            ...
        print(get_points(1000...))
        '
    """
    func_src = inspect.getsource(get_points)
    script = SUBTASK_SCRIPT_TEMPLATE.format(func_src=func_src, points_num=SUBTASK_POINTS)
    return [sys.executable, "-c", script]

#### Using Trio
Trio has
[two options](https://trio.readthedocs.io/en/stable/reference-io.html#options-for-starting-subprocesses)
to start a process, the simpler one is
[trio.run_process()](https://trio.readthedocs.io/en/stable/reference-io.html#trio.run_process).

In [4]:
!pip install trio

You should consider upgrading via the '/Users/ofedoro/src/Asynchronous-programming-with-Python/.venv/bin/python3.7 -m pip install --upgrade pip' command.


In [17]:
%%time
import os

import trio


async def main():
    points_list = []
    limiter = trio.CapacityLimiter(os.cpu_count() or 2)

    async with trio.open_nursery() as nursery:
        for _ in range(SUBTASKS_NUM):
            nursery.start_soon(run_subtask, points_list, limiter)

    return sum(points_list)


async def run_subtask(points_list, limiter):
    """Calculate the points in a subprocess."""
    async with limiter:
        completed_process = await trio.run_process(
            get_subprocess_arguments(),
            capture_stdout=True
        )
        points_list.append(int(completed_process.stdout.strip()))


print("The π is roughly equal to...")
points = trio.run(main)
π = 4 * points / POINTS
print_result(π)

The π is roughly equal to...
3.14157988
The difference is: -1.2773589793013684e-05
CPU times: user 39.3 ms, sys: 45.6 ms, total: 84.8 ms
Wall time: 9.14 s


<span style="font-size: x-large">Add your code below:</span>